In [1]:
# -- run me first --
from pprint import pprint  # for pretty printing
# display all outputs, not only last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Create dir for files
# !cd ~/work/
# !rm -rf ~/work/8_modules_and_packages_files
# !mkdir -p ~/work/8_modules_and_packages_files
print("-done-")

-done-


<center>🐍</center>

***
# 8. Модули и Пакеты
<div style="text-align: right; font-weight: bold">Aleksandr Koriagin</div>
<div style="text-align: right; font-weight: bold"><span style="color: #76CDD8;">&lt;</span>epam<span style="color: #76CDD8;">&gt;</span></div>
<div style="text-align: right; font-weight: bold">May 2020</div>
<div style="text-align: right; font-style: italic">Nizhny Novgorod</div>

***
## Оглавление<a id="0"></a>

1. [Модули](#1)
    1. Подключение модуля из стандартной библиотеки
    1. Использование псевдонимов
    1. Инструкция `from`
    1. Инструкция `from` co звёздочкой
    1. Изолированное пространство имен
    1. Создание своего модуля
    1. Именование модулей
    1. Использование атрибута `__name__`
2. [Пакеты](#2)
    1. Возможности `__init__.py`
    1. Относительный импорт
    1. Что может быть импортировано?
    1. Пути поиска модулей
    1. Полный список путей поиска
    1. Импорт и PEP8
3. [Инструменты управления пакетами](#3)
    1. Cкачивание и установка пакетов через PIP
        1. `pip` или `pip3`
    1. Venv
        1. Как работает venv?
    1. Virtualenv
    1. Setuptools и `setup.py`
        1. Примеры команд
4. [Дополнительные темы](#4)
    1. Повторная загрузка модулей
    1. Циклический (рекурсивный) импорт
    1. Динамический импорт
    1. Namespace packages
    1. Файл `__main__.py`
5. [Домашняя работа](#5)

In [ ]:
%%bash
# generate table of contents
cat ~/work/8_modules_and_packages.ipynb | grep "##" | grep -v "cat" | sed  "s/#/    /g" | tr -d '"'

***
## 1. Модули<a id="1"></a>

Для простоты понимания, под термином ***модуль*** в Python мы будем понимать файл с исходным кодом, имеющий расширение `.py`, <br>а ***пакет*** – это каталог, который может включать другие каталоги или модули.

### Подключение модуля из стандартной библиотеки

Подключить модуль можно с помощью инструкции `import`

Модуль может содержать функции, классы, объекты классов и переменные – все это будет доступно для других модулей, если правильно импортировать данный. Также в модуле может содержаться программный код, который будет выполняться, если скрипт запустить на исполнение.

In [ ]:
import this

После ключевого слова `import` указывается название модуля. 
<br>Одной инструкцией можно подключить несколько модулей, хотя этого ***не рекомендуется делать, так как это снижает читаемость кода***.

In [ ]:
import time, random

time.time()
random.random()

После импортирования модуля его название становится переменной, через которую можно получить доступ к атрибутам модуля. <br>Например, можно обратиться к константе `e`, расположенной в модуле `math`:

In [ ]:
import math
math.e

Если указанный атрибут модуля не будет найден, поднимется исключение `AttributeError`. А если не удастся найти модуль для импортирования, то `ModuleNotFoundError`.

In [ ]:
import math
math.AAAA

In [ ]:
import not_exist

Примерный аналог кода для иллюстрации

In [ ]:
%%writefile ~/work/8_modules_and_packages_files/my_module.py

# Создаем свой модуль
def say_hello():
    return "Hello!"

In [ ]:
%cd ~/work/8_modules_and_packages_files

import sys
from types import ModuleType

if 'my_module' not in sys.modules:                      # Проверка кэша (повторный импорт не происходит)
    sys.modules['my_module'] = ModuleType('my_module')  # Создание объекта модуля
    code = open('my_module.py', 'rb').read()            # Чтение исходного кода (компиляция тут опущена)
    exec(code, sys.modules['my_module'].__dict__)       # Инициализация модуля (выполнение кода в пространстве имен модуля)

module_alias = sys.modules['my_module']                 # Объект появляется в локальной области видимости
module_alias.say_hello()

### Использование псевдонимов

Если название модуля слишком длинное, или оно вам не нравится по каким-то другим причинам, то для него можно создать псевдоним, с помощью ключевого слова `as`.

In [ ]:
import math as m
m.e

Теперь доступ ко всем атрибутам модуля `math` осуществляется только с помощью переменной `m`, а переменной `math` в этой программе уже не будет.

### Инструкция `from`

Подключить определенные атрибуты модуля можно с помощью инструкции `from`. Она имеет несколько форматов:
```python
from <Название модуля> import <Атрибут 1> [ as <Псевдоним 1> ], [<Атрибут 2> [ as <Псевдоним 2> ] ...]
from <Название модуля> import *
```
Первый формат позволяет подключить из модуля только указанные вами атрибуты. Для длинных имен также можно назначить псевдоним, указав его после ключевого слова `as`.

In [ ]:
from math import e, ceil as c

e
c(4.6)

Импортируемые атрибуты можно разместить на нескольких строках, если их много, для лучшей читаемости кода:
```python
from math import (
    sin, cos,
    tan, atan
)
```

### Инструкция `from` co звёздочкой

Второй формат инструкции `from` позволяет подключить все (точнее, почти все) переменные из модуля.

***Таких импортов "со звездочкой" в большинстве случаев рекомендуется избегать (см. [PEP8](https://www.python.org/dev/peps/pep-0008/#imports)).***
> Шаблоны импортов `from <module> import *` следует избегать, так как они делают неясным то, какие имена присутствуют в глобальном пространстве имён, что вводит в заблуждение как читателей, так и многие автоматизированные средства. ...

<br>Для примера импортируем все атрибуты из модуля `sys`:

In [ ]:
from sys import *

version
version_info

Форма `from module import *` доступна только на верхнем уровне импортирующего модуля. При попытке использовать ее внутри функции возникнет исключение `SyntaxError` (еще на этапе определения этой функции).

In [ ]:
def some_func():
    from sys import *
    print(version)

some_func()

Следует заметить, что не все атрибуты будут импортированы. 
Если в модуле определена переменная `__all__` (список атрибутов, которые могут быть подключены), то будут подключены только атрибуты из этого списка. Если переменная `__all__` не определена, то будут подключены все атрибуты, не начинающиеся с нижнего подчёркивания. Кроме того, необходимо учитывать, что импортирование всех атрибутов из модуля может нарушить пространство имен главной программы, так как переменные, имеющие одинаковые имена, будут перезаписаны.

In [ ]:
%%writefile alltest.py
# file: alltest.py
# Если __all__ не задан, импортируются все имена. Это распространяется только на импорты через "*"
__all__ = ['say_something']

def say_something():
    return some_str

some_str = "Hello!"
some_dict = {"a": 1}
some_int = 42

In [ ]:
%cd ~/work/8_modules_and_packages_files

from alltest import *

say_something
say_something()
some_str       # NameError

In [ ]:
%cd ~/work/8_modules_and_packages_files

import alltest
alltest.some_str

from alltest import some_str as some_str_my
some_str_my

### Изолированное пространство имен

In [2]:
%pycat alltest.py

In [3]:
# %cd ~/work/8_modules_and_packages_files
from alltest import say_something, some_str, some_dict

# Функции модуля используют глобальные переменные модуля
some_str = "Goodbye!"
print(f"1) say_something() => {say_something()}\n")

some_dict["b"] = 2       # Не рекомендуется без причины
some_dict = {"new": 789}

import alltest
print(f"2) alltest.some_dict ==> {alltest.some_dict}\n")

alltest.some_str = "May the Force be with you"    # monkey patching
print(f"3) say_something() => {say_something()}")

1) say_something() => Hello!

2) alltest.some_dict ==> {'a': 1, 'b': 2}

3) say_something() => May the Force be with you


In [ ]:
## TODO!!!
# %cd ~/work/8_modules_and_packages_files

from alltest import say_something, some_dict
some_dict
some_dict["b"] = 2
some_dict

### Создание своего модуля

In [ ]:
%%writefile ~/work/8_modules_and_packages_files/my_simplemath.py
# file: my_simplemath.py

VAR = 'some my var'

class SomeClass:
    a = 1

some_class = SomeClass()

def add(a, b):
    return a + b

def sub(a, b):
    return a - b

def mul(a, b):
    return a * b

def div(a, b):
    return a / b

In [ ]:
%cd ~/work/8_modules_and_packages_files

import my_simplemath

my_simplemath.add(1, 2)   # function
my_simplemath.VAR         # variable
my_simplemath.SomeClass   # class
my_simplemath.some_class  # class instance

In [ ]:
%cd ~/work/8_modules_and_packages_files

from my_simplemath import sub

sub(2, 3)

### Именование модулей

На имена модулей накладываются те же требования, что и на имена переменных.

Дополнительные рекомендации:
* Имена модулей должны быть в нижнем регистре
* Cледует избегать использование в имени символы не из набора ASCII
* Можно использовать нижнее подчеркивание для обозначения "скрытых" внутренних модулей
* Следует избегать совпадения названия с модулями из стандартной библиотеки
* Модуль нельзя именовать также, как и ключевое слово (`class`, `except`, `False`, ...)


### Использование атрибута `__name__`

При импортировании модуля его код выполняется полностью, то есть, если программа что-то печатает, то при её импортировании это будет напечатано. Этого можно избежать, если проверять, запущен ли скрипт как программа, или импортирован. Это можно сделать с помощью переменной `__name__`, которая определена в любой программе, и равна `"__main__"`, если скрипт запущен в качестве главной программы, и имя, если он импортирован. Например, `mymodule.py` может выглядеть вот так:

In [ ]:
%%writefile ~/work/8_modules_and_packages_files/nametest.py
#file: nametest.py

def say_something():
    print(f"Hello! My __name__ is: '{__name__}'")

if __name__ == "__main__":
    say_something()

In [ ]:
%cd ~/work/8_modules_and_packages_files

import nametest

nametest.say_something()

In [ ]:
%run ~/work/8_modules_and_packages_files/nametest.py

***
## 2. Пакеты<a id="2"></a>

Cвязанные модули принято объединять в пакеты. Пакет в Python – это каталог, включающий в себя другие каталоги и модули, но при этом дополнительно содержащий файл `__init__.py`. Пакеты используются для формирования пространства имен, что позволяет работать с модулями через указание уровня вложенности (через точку).

Для импортирования пакетов используется тот же синтаксис, что и для работы с модулями.

Рассмотрим следующую структуру пакета:
```
fincalc/
|-- __init__.py
|-- simper.py
|-- compper.py
|-- annuity.py
```
Пакет `fincal` содержит в себе модули для работы с простыми процентами (`simper.py`), сложными процентами (`compper.py`) и аннуитетами (`annuity.py`).

Для использования фукнции из модуля работы с простыми процентами, можно использовать один из следующих вариантов:
```python
import fincalc.simper
fv = fincalc.simper.fv(pv, i, n)

import fincalc.simper as sp
fv =sp.fv(pv, i, n)

from fincalc import simper
fv = simper.fv(pv, i, n)
```
Файл `__init__.py` может быть пустым или может содержать переменную `__all__`, хранящую список модулей, который импортируется при загрузке через конструкцию
```python
from имя_пакета import *
```
Например для нашего случая содержимое `__init__.py` может быть вот таким:
```python
__all__ = ["simper", "compper", "annuity"]
```

Например, возможная структура пакета:
```
sound/                          Пакет верхнего уровня
      __init__.py               Инициализация пакета работы со звуком (sound)
      formats/                  Подпакет для конвертирования форматов файлов
              __init__.py
              wavread.py        (чтение wav)
              wavwrite.py       (запись wav)
              aiffread.py       (чтение aiff)
              aiffwrite.py      (запись aiff)
              auread.py         (чтение au)
              auwrite.py        (запись au)
              ...
      effects/                  Подпакет для звуковых эффектов
              __init__.py
              echo.py           ( эхо )
              surround.py       ( окружение )
              reverse.py        ( обращение )
              ...
      filters/                  Подпакет для фильтров
              __init__.py
              equalizer.py      ( эквалайзер )
              vocoder.py        ( вокодер ) 
              karaoke.py        ( караоке )
              ...
```

### Возможности `__init__.py`

`__init__.py` позволяет Питону интерпретировать директория как пакет. 

В общем случае файл `__init__.py` предназначен для выполнения действий по инициализации пакета, создания пространства имен для каталога и реализации поведения инструкций `from ... import *`, когда они используются для импортирования каталогов.

Когда интерпретатор Python импортрирует каталог в первый раз он автоматически запускает программный код файла `__init__.py` этого каталога.

```python
# file: mypackage/__init__.py

# Импортировать имена из подмодулей для удобства
# (для больших библиотек автоматически импортировать все подмодули нежелательно)
from mypackage.submodule import say_something 

# Ограничить доступность имен для импорта через "*"
__all__ = ['say_something']

# Добавить любой код инициализации пакета
# (определять здесь сами импортируемые объекты нежелательно)
print('Initializing package')

# Изменить пути поиска подмодулей (o_0)
__path__.append(['/path/to/directory', '.'])

# Можно оставить данный файл и вовсе пустым.
```

### Относительный импорт

При ***абсолютном*** импорте используется полный путь (от начала корневой папки проекта) к желаемому модулю.

При ***относительном*** импорте используется относительный путь (начиная с пути текущего модуля) к желаемому модулю. Есть два типа относительных импортов:
* При явном импорте используется формат `from .<модуль/пакет> import X`, где символы точки `.` показывают, на сколько директорий "вверх" нужно подняться. Одна точка `.` показывает текущую директорию, две точки `..` — на одну директорию выше и т. д.
* Неявный относительный импорт пишется так, как если бы текущая директория была частью `sys.path`. Такой тип импортов поддерживается только в Python 2.

В документации Python об относительных импортах в Python 3 написано следующее:
> Единственный приемлемый синтаксис для относительных импортов — `from .[модуль] import [имя]`. Все импорты, которые начинаются не с точки `.`, считаются абсолютными. [link](https://docs.python.org/3.0/whatsnew/3.0.html#removed-syntax)

**Абсолютные импорты предпочтительнее относительных**.
<br>Они позволяют избежать путаницы между явными и неявными импортами. Кроме того, любой скрипт с явными относительными импортами нельзя запустить напрямую

Больше информации:
* [Абсолютный и относительный импорт](https://tproger.ru/translations/guide-into-python-imports/#11)
* [Absolute vs Relative Imports in Python](https://realpython.com/absolute-vs-relative-python-imports/#relative-imports)
* [Relative imports in Python 3](https://stackoverflow.com/questions/16981921/relative-imports-in-python-3)

### Что может быть импортировано?

```python
import module
```
Чем может быть `module` в данном случае:
* файлом с исходным кодом (`module.py`), в т.ч. в ZIP-архиве
* файлом с байт-кодом (`module.pyc`), компиляция которого прошла при прошедшем ранее импорте
* пакетом (каталогом с именем `module`)
* встроенным модулем (таким как например `math`)
* динамической библиотекой (`module.so`, `module.dll`) для `CPython`
* классом `Java` для `Jython`, компонентом `.NET` для `IronPython`
* ну и в самом общем случае чем угодно (если переопределить механизм импорта) ...

### Пути поиска модулей:

1. Каталог с запускаемым файлом (в интерактивном режиме - текущий каталог)
1. Пути из переменной окружения `PYTHONPATH` (разделенные `:` или `;` в зависимости от операционной системы)
1. Каталоги стандартной библиотеки
1. Глобальный каталог `site-packages` (`pip install …`)
1. Пользовательский каталог `site-packages` (`pip install –-user …`)
1. Дополнительные каталоги `dist-packages` на Linux (`apt install …`)
1. Файлы `*.pth` в `site-packages`
1. … и другие (см. `sys.path` в случае сомнений)


**Подробне**: <br>"Python in a Nutshell", глава _"Поиск модуля в файловой системе"_ : [PDF](http://www.williamspublishing.com/PDF/978-5-6040723-8-7/part.pdf)

### Полный список путей поиска

* Поиск модулей будет производиться только в каталогах, присутствующих в списке `sys.path`
* `sys.path` формируется при старте интерпретатора по крайне запутанному сценарию в зависимости от множества факторов и параметров (например часть записей напрямую зависит от каталога установки Python, т.н. "префикса")
* `sys.path` – список изменяемый (вносимые изменения будут влиять на все последующие импорты модулей)
* Пути в списке могут как абсолютными, так и относительными (от текущего каталога)
* Наравне с каталогами в списке могут фигурировать ZIP-архивы и EGG-файлы


In [ ]:
%cd ~/work
# export PYTHONPATH='/aaa/bbb'
%env PYTHONPATH /aaa/bbb

!python3 -c "import sys; import pprint; pprint.pprint(sorted(sys.path))";

In [ ]:
import sys
pprint(sorted(sys.path))

### Импорт и PEP8

Imports should be grouped in the following order:
* Standard library imports.
* Related third party imports.
* Local application/library specific imports.

You should put a blank line between each group of imports.

https://www.python.org/dev/peps/pep-0008/#imports

**isort**: https://github.com/timothycrosley/isort

***
## 3. Инструменты управления пакетами<a id="3"></a>

### Cкачивание и установка пакетов через PIP

PIP – менеджер пакетов для Python, умеющий скачивать пакеты (как из централизованного хранилища PyPI, так и например из любых git-репозиториев) и устанавливать их. Появился как замена похожему на него инструменту easy_install. [PIP user_guide](https://pip.pypa.io/en/latest/user_guide/)

Help:
```
$ pip --help
$ pip install --help
$ pip uninstall --help
```

numpy: https://pypi.org/project/numpy/

Установить последнюю версию пакета:
```
$ pip install --upgrade numpy
```

Установить конкретную версию пакета:
```
$ pip install numpy==1.18.1
$ pip install numpy==1.*
$ pip install 'numpy>=1.0,<2.0'
```

Установить из GIT:
```
$ pip install git+https://github.com/numpy/numpy
```

Удалить пакет:
```
$ pip uninstall numpy
```

Установить список пакетов из файла:
```
$ pip install –r requirements.txt

requirements.txt:
    numpy==1.18.1
    some_other_package==1.2.3
```

Вывести список установленных пакетов и их версий:
```
$ pip freeze
$ pip list
```

Сохранить список пакетов в файл:
```
$ pip freeze > requirements.txt
```

#### `pip` или `pip3`

В зависимости от того, как установлен и настроен Python в системе, может потребоваться использовать `pip3` вместо `pip`. Чтобы проверить, какой вариант используется, надо выполнить команду `pip --version`.
```
$ pip --version
pip 20.0.2 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)
```
Если есть сомнения, то лучше использовать альтернативный вариант вызова pip:
```
$ python3 -m pip install numpy
```

### Venv

`Virtualenv` – инструмент для работы с несколькими "виртуальными окружениями", в каждом из которых установлен отдельный набор пакетов определенных версий. В последних версиях Python входит в стандартную библиотеку, ранее же был сторонним компонентом, устанавливаемым как правило средствами pip или пакетного менеджера операционной системы.

https://docs.python.org/3/library/venv.html

Установить если нет в системе:
```
$ sudo apt-get install python3-venv
```
Создать виртуальное окружение в каталоге `py3venv`
```
$ python3 -m venv py3venv
```
"Активировать" окружение (установить переменную окружения `PATH`)
```
$ source py3venv/bin/activate
```
Установить пакет:
```
(.py3venv)$ pip install numpy
```
Запустить интерпретатор:
<br>можно запускать напрямую из `bin/` без "activate"
```
(.py3venv)$ python
```
"Деактивировать" окружение:
 ```
 $ deactivate
 ```

#### Как работает venv?

Пакетные менеджеры устанавливают пакеты в каталог `site-packages`, который находится внутри каталога установки Python (т.н. "префикса"). При запуске интерпретатор Python находит этот каталог установки, пробуя следующие способы (в порядке очередности):

* По содержимому файла `pyvenv.cfg`, находимому в каталоге рядом с интерпретатором либо в каталогах верхнего уровня. Новые версии virtualenv основаны именно на этом способе.
* По значению переменной окружения `PYTHONHOME`
* По расположению ключевых файлов стандартной библиотеки (например `os.py`) в каталогах верхних уровней относительно интерпретатора
* По значению префикса, зашитому в исполняемом файле интерпретатора при компиляции

```
$ tree -a -L 4 -I "__pycache__" .py3venv/
.py3venv/
├── bin
│   ├── activate
│   ├── activate.csh
│   ├── activate.fish
│   ├── easy_install
│   ├── easy_install-3.6
│   ├── pip
│   ├── pip3
│   ├── pip3.6
│   ├── python -> python3
│   └── python3 -> /usr/bin/python3
├── include
├── lib
│   └── python3.6
│       └── site-packages
│           ├── easy_install.py
│           ├── pip
│           ├── pip-9.0.1.dist-info
│           ├── pkg_resources
│           ├── pkg_resources-0.0.0.dist-info
│           ├── setuptools
│           └── setuptools-39.0.1.dist-info
├── lib64 -> lib
├── pyvenv.cfg
└── share
    └── python-wheels
        ├── appdirs-1.4.3-py2.py3-none-any.whl
        ├── CacheControl-0.11.7-py2.py3-none-any.whl
        . . .
        ├── webencodings-0.5-py2.py3-none-any.whl
        └── wheel-0.30.0-py2.py3-none-any.whl

14 directories, 35 files
```
Что находится в этих папках?
* `bin` – файлы, которые взаимодействуют с виртуальной средой;
* `include` – С-заголовки, компилирующие пакеты Python;
* `lib` – копия версии Python вместе с папкой "site-packages", в которой установлена каждая зависимость.

Пример файла `pyvenv.cfg`:
```
$ cat .py3venv/pyvenv.cfg
home = /usr/bin
include-system-site-packages = false
version = 3.6.9
```

**Дополнительно**: [Виртуальная среда Python – Основы](https://python-scripts.com/virtualenv)

### Virtualenv

Для Python2, хотя хорошо работает и для Python3.

https://pypi.org/project/virtualenv/ ; https://virtualenv.pypa.io/en/latest/

```
$ sudo -H pip install -U virtualenv

$ virtualenv -p python3 --clear .venv
$ source .venv/bin/activate
$ pip install -U numpy
```


In [ ]:
!python3 -m pip install -U virtualenv
!virtualenv -p python3 --clear .myvenv
!. .myvenv/bin/activate && python -c "import sys, pprint; pprint.pprint(sorted(sys.path))";

### Setuptools и `setup.py`

Setuptools – "сторонний" пакет, основанный на `distutils` из стандартной библиотеки.

`setup.py` – это файл Python, который обычно сообщает вам, что модуль/пакет, который вы собираетесь установить, был упакован и распространен с помощью `Distutils`, который является стандартом для распространения модулей Python.

Это позволяет легко устанавливать пакеты Python. Часто достаточно написать `python setup.py install` и модуль сам установит.

#### Примеры команд

Установить сам setuptools если его нет по умолчанию:
```
$ pip install -U setuptools
```

Установить пакет:
```
$ python setup.py install
```

Установить пакет в режиме "разработки" (создавая символические ссылки или другим подобным образом):
```
$ python setup.py develop
```

Создать сборку в каталоге `./build/` (в т.ч. скомпилировать C-расширения):
```
$ python setup.py build
```

Запустить тесты:
```
$ python setup.py test
```

Загрузить пакет на PyPI (Python Package Index):
```
$ python setup.py upload
```

Пример `setup.py`:
```python
from setuptools import setup, find_packages
from os.path import join, dirname

setup(
    name='helloworld',
    version='1.0',
    packages=find_packages(),
    long_description=open(join(dirname(__file__), 'README.txt')).read(),
)
```
Еще пример:
```python
from setuptools import setup, find_packages

setup(
    name="mypackage",
    version="0.1",
    packages=find_packages(),
    scripts=['say_hello.py'],
    install_requires=['docutils>=0.3'],
    # метаданные для PyPI
    author="Me",
    author_email="me@example.com",
    description="This is an Example Package",
    license="PSF",
    url="http://example.com/HelloWorld/",
)
```
Еще примеры: [attrs/setup.py](https://github.com/python-attrs/attrs/blob/master/setup.py), [XlsxWriter/setup.py](https://github.com/jmcnamara/XlsxWriter/blob/master/setup.py), ...


**Дополнительно**: [Создание python-пакетов](https://klen.github.io/create-python-packages.html)

***
## 4. Дополнительные темы<a id="4"></a>

### Повторная загрузка модулей

Программный код модуля по умолчанию запускается всего один раз за все время работы программы. Чтобы принудительно повторно загрузить модуль и запустить программный код в нем, необходимо явно вызвать встроенную функцию `reload`.

Ограничения:
* заново запускает код в пространстве имен модуля (инструкции будут изменять существующий объект модуля, удаление имен произведено не будет)
* перезагрузка не транзитивна (для перезагрузки всех зависимостей модуля нужно писать рекурсивный обход вручную, исключая циклы)
* не сработает для пакетов (см. предыдущий пункт)
* не сработает с модулями расширения, написанными на C/C++
* не сработает для импортов, сделанных с помощью `from … import` (после перезагрузки их нужно будет делать повторно)

Зачем это нужно:
* для применения изменений кода при работе в интерактивном интерпретаторе (теоретически)
* для повторного выполнения кода инициализации модуля (без фактического изменения файла)

Перезагрузка модуля:
```python
import importlib

import module
. . .
. . .
importlib.reload(module)
```
Примерный аналог для иллюстрации:
```python
import module
code = open(module.__file__, 'rb').read()
exec(code, module.__dict__)
```

### Циклический (рекурсивный) импорт

In [ ]:
%cd ~/work/8_modules_and_packages_files

In [4]:
%%writefile foo.py
# file: foo.py

import bar

def func():
    print(bar.get_message())

Writing foo.py


In [5]:
%%writefile bar.py
# file: bar.py

import foo

foo.func()  # Здесь будет AttributeError

def get_message():
    return 'Hello from bar.py!'

Writing bar.py


In [7]:
!python bar.py

Traceback (most recent call last):
  File "bar.py", line 3, in <module>
    import foo
  File "C:\Users\K\Documents\epam_python\class_8_modules_packages\foo.py", line 3, in <module>
    import bar
  File "C:\Users\K\Documents\epam_python\class_8_modules_packages\bar.py", line 5, in <module>
    foo.func()  # Здесь будет AttributeError
AttributeError: module 'foo' has no attribute 'func'


Способы решить эту проблему (**помимо написания нормального кода**):

`foo.py`:
```python
def func():
    import bar
    print(bar.get_message())

func()
```

`bar.py`:
```python
def foo_func():
    import foo
    foo.func()

def get_message():
    return 'Hello from bar.py!'

foo_func()
```
Result:
```
$ python3 bar.py
    Hello from bar.py!
    Hello from bar.py!
    Hello from bar.py!
```
**Вопрос**: Почему `Hello from bar.py!` распечатан 3 раза?

### Динамический импорт

Python предлагает пакет `importlib` в качестве части стандартной библиотеки модулей. Его задача – обеспечить реализацию оператора импорта Python, а также функции `__import__()`

In [ ]:
import importlib

importlib.import_module('math')

importlib.import_module('asyncio.coroutines')

importlib.import_module('.coroutines', package='asyncio')

Example:
```python
data_path_template = "packages.model.tasks.tasks.{COUNTRY}.{TASK}.task"
for job in jobs:
    path = data_path_template.format(**job["environmentVariables"])
    tasks_module = importlib.import_module(path)
    for record in tasks_module.data:
        . . .
```

### Namespace packages

`Namespace package` – это пакет, в котором отсутствует файл `__init__.py`

Несколько пакетов с таким именем (например находящиеся в разных каталогах из `sys.path`) логически сливаются в одно пространство имен. Все подмодули такого составного пакета могут импортировать друг друга по абсолютному и относительному пути, как будто они находятся в одном обычном пакете.

Для таких пакетов конструкция `from mypackage import *` работать не будет.

Такое поведение пакетов появилось только в Python 3.

**Больше информации**: 
* [Объединение нескольких пакетов в одно пространство имен Python](https://habr.com/ru/post/458432/)
* [PEP 420 -- Implicit Namespace Packages](https://www.python.org/dev/peps/pep-0420/#id3)
* [Packaging namespace packages](https://packaging.python.org/guides/packaging-namespace-packages/)

### Файл `__main__.py`

Модуль `__main__.py` обозначает точку входа при запуске пакета на выполнение.

```
$ python3 my_program.py

$ python3 my_program_dir  # запуск my_program_dir/__main__.py
$ python3 my_program.zip  # запуск __main__.py из архива

$ python3 -m my_program   # сработает, если пакет my_program доступен для импорта 

# Примеры из стандартной библиотеки
$ python3 -m pip install django
$ python3 -m http.server  # запуск HTTP-сервера на 8000 порту для отдачи файлов из текущего каталога
```

**Больше информации**:
* [Используйте \_\_main\_\_.py](https://habr.com/ru/post/456214/)


***
#### Дополнительно:
[Модули и пакеты в Python. Глубокое погружение](https://devpractice.ru/python-modules-and-packages/#p3)

***
## 5. Домашняя работа<a id="5"></a>

### HW 1
Создайте свою любую программу содержащую несколько модулей.

Например, калькулятор:
```
code/
    - addition/
        - add.py
    - subtraction/
        - subtr.py
run_calc.py
```
`python3 run_calc.py "2+3"`

Не забывайте про `if __name__ == '__main__'`
***

In [ ]:
# Clean-up
!cd ~/work/
!rm -rf ~/work/8_modules_and_packages_files
print("-done-")

<center>🐍</center>